In [4]:
import sys
import time
from os import path
sys.path.append('/Users/linhua/PycharmProjects/Fupan')
from private import tushare_token
import tushare as ts
import sqlite3
import pandas as pd


In [5]:
# prepare Tushare data interface
ts.set_token(tushare_token.tushare_token)
pro = ts.pro_api()

In [6]:
# 获取概念股分类
concept_list_df = pro.concept()
# concept_list_df

#取5G概念明细
concept_detail_df = pro.concept_detail(id='TS271', fields='ts_code,name')
# concept_detail_df

#查询某个股票的概念
equity_concept_df = pro.concept_detail(ts_code = '300212.SZ')
# equity_concept_df

concept_detail_df

,ts_code,name
0,000413.SZ,东旭光电
1,000536.SZ,华映科技
2,002036.SZ,联创电子
3,002217.SZ,合力泰
4,002426.SZ,胜利精密
5,002456.SZ,欧菲光
6,002547.SZ,春兴精工
7,002594.SZ,比亚迪
8,002866.SZ,传艺科技
9,300083.SZ,劲胜智能


In [47]:
# Database design:
# table: concept_list
#     | concept_code | concept_name | src |

# table: concept_dashboard
#     | concept_name (1) | concept_name (2) |

# context = sqlite3.connect('/Users/linhua/PycharmProjects/Fupan/database/tushare.db')
# concept_df.to_sql('concept_list', context)
def update_concept_database(dpath):
    concept_list_df = pro.concept()
    
    concept_dashboard_code = pd.DataFrame()
    concept_dashboard_name = pd.DataFrame()
    counter = 0
    
    for idx, row in concept_list_df.iterrows():
        print(str(idx) + '  ' + str(row['name']))
        df = pro.concept_detail(id=row.code, fields='ts_code,name')
        concept_dashboard_code = pd.concat([concept_dashboard_code, df['ts_code']], axis=1)
        concept_dashboard_name = pd.concat([concept_dashboard_name, df['name']], axis=1)
        concept_dashboard_code.rename(columns={'ts_code':row['name']}, inplace=True)
        concept_dashboard_name.rename(columns={'name':row['name']}, inplace=True)
        
        counter = counter+1
        if (counter > 90):
            counter = 0
            time.sleep(60)
        
    context = sqlite3.connect(dpath)
    concept_list_df.to_sql('concept_list', context, if_exists='replace')
    concept_dashboard_code.to_sql('concept_dashboard_code', context, if_exists='replace')
    concept_dashboard_name.to_sql('concept_dashboard_name', context, if_exists='replace')
    
    return concept_list_df, concept_dashboard_code, concept_dashboard_name

clist, dbc, dbn = update_concept_database('/Users/linhua/PycharmProjects/Fupan/database/tushare.db')

0  密集调研
1  南北船合并
2  5G
3  机场
4  高价股
5  烧碱
6  AH溢价股
7  保险
8  PVC
9  啤酒
10  火电
11  银行
12  碳纤维
13  安邦系
14  特高压
15  光通信
16  草甘膦
17  高速公路
18  有色-铝
19  有色-锆
20  主题公园
21  白马股
22  雄安-地产
23  OLED
24  MSCI
25  健康中国
26  养老金持股
27  水电
28  环氧丙烷
29  PCB板
30  煤炭
31  中字头
32  血制品
33  钢铁
34  油服
35  内蒙古
36  券商
37  有色-锌
38  人民币升值受益
39  阿里巴巴概念股
40  有色-锑
41  海工装备
42  新能源整车
43  涤纶
44  振兴东北
45  有色-镁
46  油气改革
47  一带一路
48  新进指数成份股
49  玻纤
50  大消费
51  被动元件
52  北汽新能源
53  可燃冰
54  东盟博览会
55  电改
56  石墨电极
57  北京城市规划
58  特斯拉
59  赛马
60  航运
61  金改
62  超级高铁
63  农药
64  汽车整车
65  次新股
66  安徽国企改革
67  水务
68  混改
69  PD-1抑制剂
70  二胎
71  固废处理
72  医药商业
73  教育
74  军工混改
75  海底捞概念股
76  硅锰
77  高送转-预期
78  央企改革
79  有机硅
80  民营医院
81  华为产业链
82  蔚来汽车概念股
83  甲醇
84  军工研究所
85  港口
86  社保重仓
87  美团概念股
88  纯碱
89  转基因
90  旅游
91  福建自贸区
92  核电
93  乡村振兴
94  调味品
95  大飞机
96  回购
97  通用航空
98  白酒
99  汽车销售
100  重庆国企改革
101  制冷剂
102  航母
103  新零售
104  饲料
105  空铁WiFi
106  有色金属
107  养猪
108  芬太尼概念
109  业绩预增
110  参股券商
111  园林
112  宝能系
113  破净股
114  饮料
115  风电
116  粤港澳

In [8]:
concept_dashboard_name

NameError: name 'concept_dashboard_name' is not defined

In [48]:
sql_cmd = "select * from concept_dashboard_name"
context = sqlite3.connect('/Users/linhua/PycharmProjects/Fupan/database/tushare.db')
df = pd.read_sql(sql_cmd, context, index_col='index')

In [49]:
def lookup_concept(stockname):
#     counter = 0
    concept_bucket = []
    for label, content in df.iteritems():
#         print(label)
#         print(stockname)
#         print(content)
        

        if stockname in content.values:
            concept_bucket.append(label)
#         counter = counter+1
#         if counter == 1:
#             counter = 0;
#             break
    return concept_bucket

buket = lookup_concept('易华录')
print(buket)

['阿里巴巴概念股', '央企改革', '华为产业链', '回购', '物联网', '百度概念股', '无人驾驶', '雄安-智慧城市', '养老产业', '智慧停车', '北斗导航', 'PPP', '小米概念股', '数字中国', '雄安新区', '京津冀', '大数据', '云计算', '智慧城市', '智慧安防', '独角兽', '电子车牌']


In [50]:
df

,密集调研,南北船合并,5G,机场,高价股,烧碱,AH溢价股,保险,PVC,啤酒,...,创投,电子车牌,雄安-设计,工业大麻,透明工厂,人造肉,垃圾分类,科创板,科创板对标,新型病毒
index,,,,,,,,,,,,,,,,,,,,,
0,东方盛虹,乐普医疗,神州高铁,深圳机场,古井贡酒,ST宜化,海螺水泥,天茂集团,ST宜化,燕京啤酒,...,中国宝安,厦门信达,华控赛格,深大通,银轮股份,京粮控股,中国天楹,华兴源创,众合科技,海王生物
1,冀东水泥,久之洋,中兴通讯,白云机场,长春高新,新金路,None,西水股份,新金路,*ST西发,...,深圳能源,远望谷,中设股份,广州浪奇,海得控制,丰乐种业,徐工机械,睿创微纳,西部材料,丽珠集团
2,华菱钢铁,中国应急,特发信息,上海机场,五粮液,英力特,None,中国平安,英力特,兰州黄河,...,深赛格,亚联发展,苏交科,仁和药业,英威腾,登海种业,长虹美菱,天准科技,大立科技,吉林敖东
3,中航机电,中船科技,汇源通信,厦门空港,洋河股份,沈阳化工,None,中国人保,*ST盐湖,珠江啤酒,...,深圳华强,千方科技,启迪设计,恒天海龙,科华恒盛,ST天宝,中原环保,容百科技,超华科技,仁和药业
4,莱宝高科,中国船舶,风华高科,None,北方华创,航锦科技,None,新华保险,冀中能源,重庆啤酒,...,华控赛格,金溢科技,丝路视觉,通化金马,泰尔股份,美盈森,山推股份,杭可科技,辉煌科技,贝瑞基因
5,大立科技,中国动力,创维数字,None,牧原股份,中泰化学,None,中国太保,鸿达兴业,惠泉啤酒,...,特发信息,高新兴,维业股份,ST银河,科远智慧,大北农,漳州发展,光峰科技,北方华创,普洛药业
6,广联达,中船防务,航锦科技,None,凯莱英,天原集团,None,中国人寿,中泰化学,青岛啤酒,...,穗恒运A,经纬辉开,杰恩设计,德展健康,雪迪龙,双塔食品,启迪环境,澜起科技,合众思壮,岳阳兴长
7,国星光电,中国海防,富通鑫茂,None,中新赛克,世龙实业,None,None,天原集团,乐惠国际,...,华金资本,易华录,建科院,秦川机床,锐奇股份,大康农业,城发环境,中国通号,高德红外,中粮科技
8,旷达科技,中远海控,法尔胜,None,深南电路,新疆天业,None,None,新疆天业,None,...,粤电力A,万集科技,设研院,诚志股份,中威电子,金字火腿,现代投资,福光股份,大博医疗,重药控股


In [27]:
concept_detail_df.columns

Index(['ts_code', 'name'], dtype='object')